In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1" #model will be trained on GPU 1

In [ ]:
import tensorflow.keras
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant, RandomNormal
import seaborn as sns



In [ ]:
file = 'data/pantheon.txt'
data = pd.read_csv(file, sep = " " ,usecols=['zcmb', 'mb', 'dmb'])
sns.pairplot(data, diag_kind="kde")
plt.savefig("distSNPantheon.png")
data = np.sort(data.values, axis=0)
data[1047, :]

In [ ]:
# randomize = np.random.permutation(len(data.values))
# randomize
shuffle = [x for x in range(len(data)) if x%2==1]
even = [x for x in range(len(data)) if x%2==0]
shuffle.extend(even)
np.max(shuffle)

Let's prepare our input data. 

In [ ]:
print(data[1047, 0])
data = data[shuffle]

In [ ]:
X = data[:, 0]
y = data[:, 1:]
X.shape, y.shape

In [ ]:
split = 0.8
ntrain = int(split * len(X))
indx = [ntrain]
X_train, X_test = np.split(X, indx)
y_train, y_test = np.split(y, indx)

In [ ]:
# scaler = StandardScaler()
# # scaler = MinMaxScaler(feature_range=(-1,1))
# # fit scaler on data
# scaler.fit(X.reshape(-1,1))
# # apply transform
# X = scaler.transform(X.reshape(-1,1))
# # X

In [ ]:
def model(input_z):
    efirst = Dense(100, activation='relu', input_shape=(1,))(input_z)
    ehidden2 = Dense(100, activation='relu')(efirst)
    ehidden3 = Dense(100, activation='relu')(ehidden2)
    elast = Dense(2, activation='linear')(ehidden3)
       
    return elast

In [ ]:
batch_size = 32
epochs = 100
input_z = Input(shape = (1,))

In [ ]:
snmodel = Model(input_z, model(input_z))
snmodel.compile(loss='mean_squared_error', optimizer = 'adam')

In [ ]:
snmodel.summary()

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.001,
                                   patience=10,
                                   restore_best_weights=True)]

In [ ]:
snmodel_train = snmodel.fit(X_train, y_train, batch_size=batch_size,epochs=epochs,verbose=1, 
                            validation_data=(X_test, y_test))

In [ ]:
plt.plot(snmodel_train.history['loss'], color='r', )
plt.plot(snmodel_train.history['val_loss'], color='g')
# plt.title('model loss function')
plt.ylabel('MSE')
plt.xlabel('Epochs')
plt.legend(['train', 'val'], loc='upper left')
# plt.savefig("loss_sn.png", dpi=200)

In [ ]:
pred = snmodel.predict(X_test)
pred.shape

In [ ]:
pred

In [ ]:
r = np.random.uniform(0, 2.3, size=1000)
# rr = scaler.transform(r.reshape(-1,1))
# X = scaler.inverse_transform(X)
# X
print(np.max(X), len(X), X[1047])

In [ ]:
pred_random = snmodel.predict(r)
# pred_random

In [ ]:
plt.errorbar(X, y[:,0], y[:,1], fmt='g+', markersize=10, label='Observations', alpha=0.6)
plt.errorbar(r, pred_random[:,0], pred_random[:,1], fmt='r*', markersize=3, label='Synthetic', alpha=0.05)
plt.xlabel("Redshift z")
plt.ylabel("$D_L(z)$")

plt.xlim(0,2.3)
plt.legend()
# plt.savefig("syntheticSN.png", dpi=300)

In [ ]:
# Cosmological constants
Om = 0.27

In [ ]:
def Hlcdm(z, H0=73.24):
    return H0 * np.sqrt(Om*(1+z)**3 + 1 - Om)

In [ ]:
# z = np.linspace(0, 2.2, 1000)
# # plt.scatter(data[:, 0], data[:, 1], c='g')
# yupp = pred_random[:,0]+pred_random[:,1]
# ylow = pred_random[:,0]-pred_random[:,1]
# plt.errorbar(X, y[:,0], y[:,1], fmt='g.', markersize=10, label='Observations')
# plt.errorbar(r, pred_random[:,0], pred_random[:,1], fmt='r.', markersize=1, label='Synthetic data with errors', alpha=0.01, )
# plt.plot(z, Hlcdm(z), label='$\Lambda CDM$ $H_0 = 73.24$', c='k')
# plt.plot(z, Hlcdm(z, H0=64.4), label='$\Lambda CDM$ $H_0 = 64.4$', c='b')
# plt.xlabel("Redshift z")
# plt.ylabel("$H(z)$")
# plt.xlim(0,2.2)
# plt.legend()

# plt.savefig("SyntheticHD.png",dpi=800)

In [ ]:
# pred_random.shape, r.shape

In [ ]:
# mookHz = np.concatenate((r.reshape(-1,1), pred_random), axis=1)
# np.savetxt("autoEncoderHz.dat",mookHz, delimiter=" ")